In [347]:
import os
import pandas as pd
import dotenv
dotenv.load_dotenv(dotenv.find_dotenv())
import psycopg2
import time
from sqlalchemy import create_engine
import json
from unidecode import unidecode
import re
import numpy as np

In [348]:
def connect_db():
    try:
        alchemy_url = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
            user=os.getenv('login_sql_tech'),
            password=os.getenv('senha_sql_tech'),
            host=os.getenv('host_sql_tech'),
            port=os.getenv('port_sql_tech'),
            database=os.getenv('database_sql_tech')
        )
        return create_engine(alchemy_url)
    except:
        print("Erro ao conectar ao banco de dados")
        return False

In [349]:
engine = connect_db()

In [350]:
dataFrame = pd.read_sql_query(
    "SELECT * FROM clientes",
    con = engine
    )

In [351]:
dataFrame.head()

,id_contrato,nome,codigo_cliente,documento,sexo,estado_civil,data_nascimento,tipo,endereco,data_contrato,...,valor_contrato,valor_entrada,valor_pago_parcelas,data_atualizacao,valor_encargos,valor_iptu,codigo_unidade,id_emp,cep,score
0,1324.0,ADRIANA CONEJO,2116.0,052.961.739-02,F,Solteiro(a),1986-07-28,Física,"PAULO ROBERTO ANASTACIO ,377,PARANAGUAMIRIM ,J...",2019-10-23,...,88683.53,0.00,0.00,2022-08-03,5250.62,0.00,1591.0,blv,89.231-795,9999.00
1,4173.0,JOHNSON MAKENRO JULIEN,51121.0,800.582.619-23,M,Casado(a),1989-07-07,Física,"RUA 448,650,MORRETES,ITAPEMASC",2021-12-06,...,195900.00,19590.00,13065.29,2022-08-03,0.00,0.00,4335.0,cmv,88.220-000,995.83
2,2125.0,JOSIANE ARTUSO DIAS,50717.0,010.327.499-54,F,Casado(a),1987-12-17,Física,"DOM ALVARO NUNES CABEZA DE VACA,485,BATEL,GUAR...",2020-08-17,...,65900.00,3000.00,20849.81,2022-08-03,0.00,1000.00,2779.0,crj_orb,85.015-310,1000.00
3,335.0,ULISSES FERNANDO ZOLLNER,903.0,010.146.079-14,M,Divorciado(a),1987-02-24,Física,"Gilmar Cesar Piekarski,30,Tatuquara,CURITIBAPR",2018-02-01,...,64592.88,3391.08,35314.23,2022-08-03,319.21,874.49,389.0,mar,81.470-035,757.11
4,4544.0,MICHEL RODRIGO DOS SANTOS,51695.0,387.037.838-71,M,Casado(a),1992-04-30,Física,"MATUSALEM GONÇALVES PINHEIRO,83,PARANAGUAMIRIM...",2022-06-06,...,74900.00,4165.94,0.00,2022-08-03,0.00,0.00,5631.0,jcr,89.231-603,1000.00


In [352]:
dataFrame.drop(
    ['id_contrato',
    'nome',
    'codigo_cliente',
    'endereco',
    'codigo_empreendimento',
    'nome_empreendimento',
    'imobiliaria',
    'lote',
    'quadra',
    'nome_spe',
    'telefone',
    'documento',
    'data_contrato',
    'data_status',
    'ltv',
    'rg',
    'cep',
    'nacionalidade',
    'id_emp',
    'area'], 
    axis=1, 
    inplace=True)

dataFrame

,sexo,estado_civil,data_nascimento,tipo,inadimplente,matricula,numero_aditivos,status,tier,valor_contrato,valor_entrada,valor_pago_parcelas,data_atualizacao,valor_encargos,valor_iptu,codigo_unidade,score
0,F,Solteiro(a),1986-07-28,Física,False,21.224,0.0,Distratado,,88683.53,0.00,0.00,2022-08-03,5250.62,0.00,1591.0,9999.00
1,M,Casado(a),1989-07-07,Física,False,62026,0.0,Ativo,B,195900.00,19590.00,13065.29,2022-08-03,0.00,0.00,4335.0,995.83
2,F,Casado(a),1987-12-17,Física,False,0,0.0,Ativo,B+,65900.00,3000.00,20849.81,2022-08-03,0.00,1000.00,2779.0,1000.00
3,M,Divorciado(a),1987-02-24,Física,False,0,2.0,Ativo,C,64592.88,3391.08,35314.23,2022-08-03,319.21,874.49,389.0,757.11
4,M,Casado(a),1992-04-30,Física,False,32345,0.0,Ativo,C,74900.00,4165.94,0.00,2022-08-03,0.00,0.00,5631.0,1000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3958,M,Casado(a),1979-03-26,Física,False,60.494,0.0,Distratado,,65310.11,2285.85,0.00,2022-08-03,8029.43,0.00,367.0,9999.00
3959,M,Solteiro(a),1996-05-17,Física,False,53.683,0.0,Distratado,,72900.00,3000.00,5812.21,2022-08-03,2028.97,0.00,592.0,9999.00
3960,M,Solteiro(a),1981-12-11,Física,True,61838,0.0,Ativo,D,67900.00,3617.97,0.00,2022-08-03,296.17,0.00,1227.0,311.11
3961,M,Solteiro(a),1996-04-27,Física,False,61821,0.0,Ativo,C,67900.00,3600.00,1493.02,2022-08-03,74.15,0.00,1217.0,787.04


In [357]:
# dataFrame.tipo.unique()
# dataFrame.estado_civil.unique()

1    3502
2     461
Name: tipo, dtype: int64

In [354]:
#tratando os valores
dict_sexo = {
    'M' : 1,
    'F' : 2
}
dict_tier = {
    '   ' : 0,
    'A  ': 1,
    'A+ ': 2,
    'B  ': 3,
    'B+ ': 4,
    'C  ': 5,
    'D  ' : 6,
    'D+ ' : 7,
    'E  ' : 8,
    'F  ' : 9,
    'S  ' : 10
}
dict_estado_civil = {
    '' : 0,
    'Solteiro(a)' : 1,
    'Casado(a)' : 2,
    'Divorciado(a)' : 3,
    'Separado(a) Judicialmente' : 4,
    'Viúvo(a)' : 5,
    'União Estável' : 6
}
dict_tipo = {
    'Física' : 1,
    'Jurídica' : 2
}
dict_inadimplente = {
    False : 1,
    True : 2
}


dataFrame.sexo = dataFrame.sexo.map(dict_sexo)
dataFrame.tier = dataFrame.tier.map(dict_tier)
dataFrame.estado_civil = dataFrame.estado_civil.map(dict_estado_civil)
dataFrame.tipo = dataFrame.tipo.map(dict_tipo)
dataFrame.inadimplente = dataFrame.inadimplente.map(dict_inadimplente)

In [367]:
from datetime import datetime

dataFrame.data_nascimento.unique()

dataFrame['idade'] = datetime.now().year - dataFrame.data_nascimento.year
# dataFrame['idade'] = dt.datetime.now() - dt.datetime.fromtimestamp(dataFrame['data_nascimento'])
# dataFrame['idade'] = dt.datetime.timestamp(dt.datetime.now()) - dataFrame['data_nascimento']
# dataFrame.data_nascimento.apply(dt.datetime.date(dataFrame.data_nascimento))

AttributeError: 'Series' object has no attribute 'year'

In [ ]:
dataFrame

,sexo,estado_civil,data_nascimento,tipo,inadimplente,matricula,numero_aditivos,status,tier,valor_contrato,valor_entrada,valor_pago_parcelas,data_atualizacao,valor_encargos,valor_iptu,codigo_unidade,score
0,2.0,1,1986-07-28,1,1,21.224,0.0,Distratado,0,88683.53,0.00,0.00,2022-08-03,5250.62,0.00,1591.0,9999.00
1,1.0,2,1989-07-07,1,1,62026,0.0,Ativo,3,195900.00,19590.00,13065.29,2022-08-03,0.00,0.00,4335.0,995.83
2,2.0,2,1987-12-17,1,1,0,0.0,Ativo,4,65900.00,3000.00,20849.81,2022-08-03,0.00,1000.00,2779.0,1000.00
3,1.0,3,1987-02-24,1,1,0,2.0,Ativo,5,64592.88,3391.08,35314.23,2022-08-03,319.21,874.49,389.0,757.11
4,1.0,2,1992-04-30,1,1,32345,0.0,Ativo,5,74900.00,4165.94,0.00,2022-08-03,0.00,0.00,5631.0,1000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3958,1.0,2,1979-03-26,1,1,60.494,0.0,Distratado,0,65310.11,2285.85,0.00,2022-08-03,8029.43,0.00,367.0,9999.00
3959,1.0,1,1996-05-17,1,1,53.683,0.0,Distratado,0,72900.00,3000.00,5812.21,2022-08-03,2028.97,0.00,592.0,9999.00
3960,1.0,1,1981-12-11,1,2,61838,0.0,Ativo,6,67900.00,3617.97,0.00,2022-08-03,296.17,0.00,1227.0,311.11
3961,1.0,1,1996-04-27,1,1,61821,0.0,Ativo,5,67900.00,3600.00,1493.02,2022-08-03,74.15,0.00,1217.0,787.04


In [ ]:
dataFrame.status.value_counts()

Ativo         2525
Distratado     944
Quitado        494
Name: status, dtype: int64

In [ ]:
dataFrame_ativos = dataFrame[dataFrame.status == 'Ativo']
dataFrame_ativos

,sexo,estado_civil,data_nascimento,tipo,inadimplente,matricula,numero_aditivos,status,tier,valor_contrato,valor_entrada,valor_pago_parcelas,data_atualizacao,valor_encargos,valor_iptu,codigo_unidade,score
1,1.0,2,1989-07-07,1,1,62026,0.0,Ativo,3,195900.00,19590.00,13065.29,2022-08-03,0.00,0.00,4335.0,995.83
2,2.0,2,1987-12-17,1,1,0,0.0,Ativo,4,65900.00,3000.00,20849.81,2022-08-03,0.00,1000.00,2779.0,1000.00
3,1.0,3,1987-02-24,1,1,0,2.0,Ativo,5,64592.88,3391.08,35314.23,2022-08-03,319.21,874.49,389.0,757.11
4,1.0,2,1992-04-30,1,1,32345,0.0,Ativo,5,74900.00,4165.94,0.00,2022-08-03,0.00,0.00,5631.0,1000.00
5,2.0,1,1987-12-29,1,1,13960,0.0,Ativo,3,35900.00,3000.00,1527.88,2022-08-03,0.00,0.00,5928.0,1000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,2.0,2,1964-11-22,1,1,0,0.0,Ativo,2,53555.64,0.00,37415.75,2022-08-03,64.96,663.58,306.0,988.69
3957,2.0,1,1985-02-12,1,1,0,0.0,Ativo,5,63908.25,0.00,45154.72,2022-08-03,604.99,859.89,321.0,791.27
3960,1.0,1,1981-12-11,1,2,61838,0.0,Ativo,6,67900.00,3617.97,0.00,2022-08-03,296.17,0.00,1227.0,311.11
3961,1.0,1,1996-04-27,1,1,61821,0.0,Ativo,5,67900.00,3600.00,1493.02,2022-08-03,74.15,0.00,1217.0,787.04


In [ ]:
#Removendo os contratos ativos
dataFrame.drop(dataFrame[dataFrame.status == 'Ativo'].index, inplace=True)

dataFrame.status.value_counts()

Distratado    944
Quitado       494
Name: status, dtype: int64